In [ ]:
from api_caller import evaluate_answer_generator

In [2]:
from langchain_core.language_models import BaseChatModel
import datasets
import json
from tqdm import tqdm
from typing import Optional

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    output_file: str,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    # try:  # load previous generations if they exist
    #     with open(output_file, "r") as f:
    #         outputs = json.load(f)
    # except:
    outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        # answer, relevant_docs = evaluate_answer_generator(question)
        answer = evaluate_answer_generator(question)
         # Convert `ScoredVector` objects to serializable dictionaries
        # retrieved_docs = [
        #     {
        #         "id": doc.id,  # Document ID
        #         "score": doc.score,  # Similarity score
        #         "metadata": doc.metadata,  # Metadata from Pinecone
        #     }
        #     for doc in relevant_docs
        # ]

        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "context": example["content"],
            "generated_answer": answer,
            # "retrieved_docs": retrieved_docs,
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f, indent=4, ensure_ascii=False)

/Users/g7199/anaconda3/envs/capstone/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference context that gets a relevant information, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"{{an integer number between 1 and 5}} [RESULT] Feedback: {{write a feedback for criteria}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Context (Score 5):
{context}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference context?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)
   

In [ ]:
from langchain.chat_models import ChatOpenAI

OPENAI_API_KEY ='OPENAI_API_KEY'

eval_chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY, max_tokens = 1)
evaluator_name = "GPT4o-mini"


def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            context=experiment["context"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        score = 100
        def getIntegerScore( num_retries = 3 ):
          for attempt_no in range(num_retries):
              try:
                  return int(eval_result.content.split("[RESULT]")[0])
              except ValueError as error:
                  if attempt_no < (num_retries - 1):
                      print("Error: Invalid output")
                  else:
                      raise error
        score = getIntegerScore()

        experiment[f"eval_score_{evaluator_name}"] = score
        print(score)
        # experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

In [11]:
answers

NameError: name 'answers' is not defined

In [13]:

import os
if not os.path.exists("./output"):
    os.mkdir("./output")


GENERATOR_NAME = "EEVE-Korean-Instruct-10.8B-v1.0-GGUF:Q4_K_M"
from datasets import load_dataset

# Load the dataset from a local JSON file
eval_dataset = load_dataset("json", data_files="eval_dataset_final.json")

# Access the Dataset object
eval_dataset = eval_dataset["train"] # The loaded file is stored in the "train" split

settings_name = f"chunk_embeddings_rerank:True_reader-model:{GENERATOR_NAME}"
output_file_name = f"./output/rag_{settings_name}.json"

print(f"Running evaluation for {settings_name}:")

print("Loading knowledge base embeddings...")

print("Running RAG...")

# run_rag_tests(
#     eval_dataset=eval_dataset,
#     output_file=output_file_name,
#     verbose=False,
#     test_settings=settings_name,
# )

print("Running evaluation...")
evaluate_answers(
    output_file_name,
    eval_chat_model,
    evaluator_name,
    evaluation_prompt_template,
    )

Running evaluation for chunk_embeddings_rerank:True_reader-model:EEVE-Korean-Instruct-10.8B-v1.0-GGUF:Q4_K_M:
Loading knowledge base embeddings...
Running RAG...
Running evaluation...


  1%|          | 1/159 [00:00<02:10,  1.21it/s]

4


  1%|▏         | 2/159 [00:01<01:29,  1.76it/s]

5


...

1


 99%|█████████▉| 158/159 [01:23<00:00,  2.06it/s]

5


100%|██████████| 159/159 [01:23<00:00,  1.89it/s]

4


In [14]:
eval_res = load_dataset("json", data_files="output/rag_chunk_embeddings_rerank:True_reader-model:EEVE-Korean-Instruct-10.8B-v1.0-GGUF:Q4_K_M.json")

Generating train split: 159 examples [00:00, 5855.55 examples/s]


In [21]:
a = 0
for i in eval_res['train']:
    a += i['eval_score_GPT4o-mini']

print(a/159)

3.50314465408805


In [19]:
eval_res

DatasetDict({
    train: Dataset({
        features: ['question', 'true_answer', 'context', 'generated_answer', 'test_settings', 'eval_score_GPT4o-mini'],
        num_rows: 159
    })
})